# Siamese Neural Network

## 1. Khái niệm
Siamese Neural Network (SNN) là một kiến trúc mạng nơ-ron chứa hai hoặc nhiều mạng con giống hệt nhau. “Giống hệt nhau” ở đây có nghĩa là, chúng có cùng cấu hình với cùng thông số và trọng số. Việc cập nhật các thông số được phản ánh đồng thời trên cả hai mạng con của nó.

SNN được sử dụng để tìm sự giống nhau của các dữ liệu đầu (Input Data) vào bằng cách so sánh các vectơ đặc trưng của chúng. Thông thường, một mạng nơ-ron học cách để dự đoán các lớp của một bài toán. Nếu muốn thêm hay bớt các lớp mới, chúng ta phải cập nhật (huấn luyện) lại mạng nơ-ron trên toàn bộ tập dữ liệu (cả dữ liệu mới và cũ). Ngoài ra, các mạng nơ-ron sâu cần một khối lượng lớn dữ liệu để có thể huấn luyện chúng. SNN, theo một cách khác, học cách tìm ra sự giống nhau giữa các Input Data. Vì vậy, nó cho phép chúng ta phân loại các lớp dữ liệu mới mà không cần huấn luyện lại mạng nơ-ron.

## 2. Luồng hoạt động
Luồng làm việc của SNN như sau:

- Chọn một cặp Input Data (trong phạm vi bài này là ảnh) được chọn từ dataset.

- Đưa mỗi ảnh qua mỗi Sub-network của SNN để xử lý. Output của các Sub-networks là một Embedding vector.

- Tính toán khoảng cách Euclidean giữa 2 Embedding vectors đó.

- Một Sigmoid Function có thể được áp trên khoảng cách để đưa ra giá trị Score trong đoạn [0,1], thể hiện mức độ giống nhau giữa 2 Embedding vectors. Score càng gần 1 thì 2 vectors càng giống nhau và ngược lại.

## 3. Ưu điểm và nhược điểm

SNN có một số ưu điểm nổi bật như sau:

- Lượng dữ liệu cần thiết để huấn luyện SNN là rất ít. Chỉ cần vài Samples là đủ (1-5 samples) huấn luyện SNN. Phương pháp mà nó sử dụng ở đây là One-Shot Learning hoặc Few-Shot Learning. Chính vì cần ít dữ liệu huấn luyện như vậy nên chúng ta cũng không lo lắng việc dữ liệu bị mất cân bằng (Image Imbalance).

- Khả năng kết hợp với các bộ phân loại khác cao. Do cơ chế học của SNN khác biệt với các bộ phân lớp thông thường khác, nên chúng ta hoàn toàn có thể kết hợp chúng lại với nhau. Việc làm này thường cho ra kết quả tốt hơn.

- Học từ sự tương đồng về ngữ nghĩa: SNN tập trung vào việc học các Features ở các lớp sâu hơn, nơi mà các Features giống nhau được đặt gần nhau. Do đó, nó có thể hiểu được phần nào sự tương đồng về ngữ nghĩa của các Input Data.

SNN cũng có những nhược điểm sau:

- Thời gian huấn luyện lâu hơn. SNN học theo từng cặp đôi một với nhau nên khả năng học của nó chậm hơn các NN khác.

- Không thể hiện xác suất mỗi lớp trong Output. SNN chỉ đưa đưa 1 giá trị Score trong đoạn [0,1], thể hiện sự giống nhau giữa 2 Input Data. Score càng gần 1 thì 2 Input Data càng giống nhau và ngược lại.

## Thực nghiệm


In [ ]:
class SiameseDataset():
    def __init__(self,training_csv=None,training_dir=None,transform=None):
        # used to prepare the labels and images path
        self.train_df=pd.read_csv(training_csv)
        self.train_df.columns =["image1","image2","label"]
        self.train_dir = training_dir
        self.transform = transform

    def __getitem__(self,index):
        # getting the image path
        image1_path=os.path.join(self.train_dir,self.train_df.iat[index,0])
        image2_path=os.path.join(self.train_dir,self.train_df.iat[index,1])
        # Loading the image
        img1 = Image.open(image1_path)
        img2 = Image.open(image2_path)
        img1 = img0.convert("L")
        img2 = img1.convert("L")
        # Apply image transformations
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2 , th.from_numpy(np.array([int(self.train_df.iat[index,2])],dtype=np.float32))
    def __len__(self):
        return len(self.train_df)

# Load the the dataset from raw image folders
siamese_dataset = SiameseDataset(training_csv,training_dir,
                                        transform=transforms.Compose([transforms.Resize((105,105)),
                                                                      transforms.ToTensor()
                                                                      ])
                                       )

# Load the dataset as pytorch tensors using dataloader
train_dataloader = DataLoader(
    siamese_dataset, shuffle=True, num_workers=8, batch_size=config.batch_size
)


In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Setting up the Sequential of CNN Layers
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=11,stride=1),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
            nn.MaxPool2d(3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5,stride=1,padding=2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=0.3),

            nn.Conv2d(256,384 , kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384,256 , kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=0.3),
        )
        # Defining the fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(30976, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5),

            nn.Linear(1024, 128),
            nn.ReLU(inplace=True),

            nn.Linear(128,2))

    def forward_once(self, x):
        # Forward pass
        output = self.cnn(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output

    def forward(self, input1, input2):
        # forward pass of input 1
        output1 = self.forward_once(input1)
        # forward pass of input 2
        output2 = self.forward_once(input2)
        return output1, output2

In [ ]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    """

    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, x0, x1, y):
        # euclidian distance
        diff = x0 - x1
        dist_sq = torch.sum(torch.pow(diff, 2), 1)
        dist = torch.sqrt(dist_sq)

        mdist = self.margin - dist
        dist = torch.clamp(mdist, min=0.0)
        loss = y * dist_sq + (1 - y) * torch.pow(dist, 2)
        loss = torch.sum(loss) / 2.0 / x0.size()[0]
        return loss

In [ ]:
# Declare Siamese Network
net = SiameseNetwork().cuda()
# Decalre Loss Function
criterion = ContrastiveLoss()
# Declare Optimizer
optimizer = th.optim.Adam(net.parameters(), lr=1e-3, weight_decay=0.0005)
#train the model
def train():
    loss=[]
    counter=[]
    iteration_number = 0
    for epoch in range(1,config.epochs):
        for i, data in enumerate(train_dataloader,0):
            img0, img1 , label = data
            img0, img1 , label = img0.cuda(), img1.cuda() , label.cuda()
            optimizer.zero_grad()
            output1,output2 = net(img0,img1)
            loss_contrastive = criterion(output1,output2,label)
            loss_contrastive.backward()
            optimizer.step()
        print("Epoch {}\n Current loss {}\n".format(epoch,loss_contrastive.item()))
        iteration_number += 10
        counter.append(iteration_number)
        loss.append(loss_contrastive.item())
    show_plot(counter, loss)
    return net
#set the device to cuda
device = torch.device('cuda' if th.cuda.is_available() else 'cpu')
model = train()
torch.save(model.state_dict(), "output/model.pt")
print("Model Saved Successfully")